### In dit notebook worden de LLIE-methoden getest om data te genereren. Hierna worden er voorspellingen gemaakt met het valdetectiemodel om metrics en inzichten te verzamelen.

#### Onderstaande code laadt het valdetectiemodel in, zodat hier later mee gewerkt kan worden.

In [32]:
from datetime import datetime
from urllib.request import urlopen

import numpy as np
import PIL.Image
import tensorflow.lite as tflite


MODEL_FILENAME = 'model.tflite'
LABELS_FILENAME = 'labels.txt'

od_model = None
labels = None


class ObjectDetection:
    OUTPUT_TENSOR_NAMES = ['detected_boxes', 'detected_scores', 'detected_classes']

    def __init__(self, model_filename):
        self._interpreter = tflite.Interpreter(model_path=model_filename)
        self._interpreter.allocate_tensors()

        input_details = self._interpreter.get_input_details()
        output_details = self._interpreter.get_output_details()
        assert len(input_details) == 1
        self._input_index = input_details[0]['index']

        # Get dictionary with output details {name: index}
        output_name_index = {d['name']: d['index'] for d in output_details}
        self._output_indexes = [output_name_index[name] for name in self.OUTPUT_TENSOR_NAMES]

        self._input_size = int(input_details[0]['shape'][1])

    def predict_image(self, image):
        image = image.convert('RGB') if image.mode != 'RGB' else image
        image = image.resize((self._input_size, self._input_size))

        input_array = np.array(image, dtype=np.float32)[np.newaxis, :, :, :]
        self._interpreter.set_tensor(self._input_index, input_array)
        self._interpreter.invoke()

        outputs = [self._interpreter.get_tensor(i) for i in self._output_indexes]
        return outputs


def initialize():
    global od_model
    od_model = ObjectDetection(MODEL_FILENAME)
    global labels
    with open(LABELS_FILENAME) as f:
        labels = [label.strip() for label in f.readlines()]


def predict_url(image_url):
    with urlopen(image_url) as binary:
        image = PIL.Image.open(binary)
        return predict_image(image)


def predict_image(image):
    assert od_model is not None
    assert labels is not None

    predictions = od_model.predict_image(image)

    predictions = [{'probability': round(float(p[1]), 8),
                    'tagId': int(p[2]),
                    'tagName': labels[p[2]],
                    'boundingBox': {
                        'left': round(float(p[0][0]), 8),
                        'top': round(float(p[0][1]), 8),
                        'width': round(float(p[0][2] - p[0][0]), 8),
                        'height': round(float(p[0][3] - p[0][1]), 8)
                        }
                    } for p in zip(*predictions)]

    #best_prediction = max(predictions, key=lambda p: p['probability'])
    response = {'id': '', 'project': '', 'iteration': '', 'created': datetime.utcnow().isoformat(),
                'predictions': predictions}

    # print("Results: " + str(response))
    return response


#### Onderstaande code geeft de logica voor het voorspellen, berekenen van de metrics, en het tekenen van de bounding boxes.

In [6]:
initialize()

import os
from datetime import datetime
import PIL.Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.pyplot as plt

def show_image_matplotlib(image, title=""):
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(title)
    plt.axis("off")
    plt.tight_layout()
    plt.show()

def draw_bounding_box_with_title(image, bbox, predicted_label, probability, title=""):
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Afmetingen in pixels berekenen
    width, height = image.size
    left = bbox['left'] * width
    top = bbox['top'] * height
    box_width = bbox['width'] * width
    box_height = bbox['height'] * height

    # Bounding box tekenen
    rect = patches.Rectangle((left, top), box_width, box_height,
                             linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)

    # Tekst met label + probability op bounding box zetten
    label_text = f"{predicted_label} ({probability:.2f})"
    ax.text(left, top - 10, label_text, color='white', fontsize=12,
            backgroundcolor='red')

    # Titel van het figuur instellen
    ax.set_title(title, fontsize=14)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


def predict_falls(base_path, THRESHOLD):
    import time
    
    start_time = time.time()
    total_predictions = 0
    
    TP = 0  # true positives
    FP = 0  # false positives
    TN = 0  # true negatives
    FN = 0  # false negatives

    POSE_LABELS = ["laying", "sitting", "standing", "bending"]

    def is_laying_true(filename):
        if "_not_laying" in filename:
            return False
        elif "_laying" in filename:
            return True

    def is_laying_pred(prediction):
        return (
            prediction['tagName'] == "laying"
            and prediction['probability'] >= THRESHOLD
        )

    for i in range(1, 32):
        room_path = os.path.join(base_path, f"room{i}")
        if not os.path.isdir(room_path):
            print(f"{room_path} bestaat niet, overslaan...")
            continue

        for filename in os.listdir(room_path):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue  # skip geen afbeeldingen

            filepath = os.path.join(room_path, filename)
            image = PIL.Image.open(filepath)

            try:
                result = predict_image(image)
                total_predictions += 1
            except Exception as e:
                print(f"Fout bij voorspellen van {filepath}: {e}")
                continue

            predictions = result['predictions']
            if not predictions:
                print(f"Geen voorspelling voor {filename}")
                continue

            pose_predictions = [p for p in predictions if p['tagName'] in POSE_LABELS]

            if pose_predictions:
                best_pred = max(pose_predictions, key=lambda p: p['probability'])
                predicted = is_laying_pred(best_pred)
            else:
                predicted = False
                
            actual = is_laying_true(filename)

            if actual and predicted:
                TP += 1
            elif not actual and predicted:
                FP += 1
                print(f"[FALSE POSITIVE] Room: room{i}, File: {filename}")
                draw_bounding_box_with_title(
                    image,
                    bbox=best_pred['boundingBox'],
                    predicted_label=best_pred['tagName'],
                    probability=best_pred['probability'],
                    title=f"False Positive - room{i}/{filename}"
                )
            elif not actual and not predicted:
                TN += 1
            elif actual and not predicted:
                FN += 1
                print(f"[FALSE NEGATIVE] Room: room{i}, File: {filename}")
                draw_bounding_box_with_title(
                    image,
                    bbox=best_pred['boundingBox'],
                    predicted_label=best_pred['tagName'],
                    probability=best_pred['probability'],
                    title=f"False Positive - room{i}/{filename}"
                )

    end_time = time.time()
    total_time = end_time - start_time
    avg_time_per_prediction = total_time / total_predictions if total_predictions > 0 else 0

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    print("\n=== Resultaten ===")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"\n=== Timing ===")
    print(f"Totale tijd: {total_time:.2f} seconden")
    print(f"Aantal voorspellingen: {total_predictions}")
    print(f"Gemiddelde tijd per voorspelling: {avg_time_per_prediction:.4f} seconden")


#### Baseline waarden, om later op te kunnen vergelijken. elke predict_falls() is steeds 2 keer uitgevoerd, met threshold 0.05 en 0.5. Omdat het ingeladen van alle afbeeldingen in de notebook veel opslag en overzicht kost, zijn niet alle resultaten bewaard gebleven.

In [ ]:
predict_falls("data-filtered-rooms", 0.5)

#### Traditionele methoden setup, beginnend met LIME https://github.com/aeinrw/LIME/blob/master/LIME_CLI.py

In [ ]:
import os
import numpy as np
from scipy import fft
from skimage import io, exposure, img_as_ubyte, img_as_float
from tqdm import trange
from PIL import Image

# ------------------ LIME IMPLEMENTATIE ------------------ #
def firstOrderDerivative(n, k=1):
    return np.eye(n) * (-1) + np.eye(n, k=k)

def toeplitizMatrix(n, row):
    vecDD = np.zeros(n)
    vecDD[0] = 4
    vecDD[1] = -1
    vecDD[row] = -1
    vecDD[-1] = -1
    vecDD[-row] = -1
    return vecDD

def vectorize(matrix):
    return matrix.T.ravel()

def reshape(vector, row, col):
    return vector.reshape((row, col), order='F')

class LIME:
    def __init__(self, iterations=10, alpha=2, rho=2, gamma=0.7, strategy=2):
        self.iterations = iterations
        self.alpha = alpha
        self.rho = rho
        self.gamma = gamma
        self.strategy = strategy

    def load(self, imgPath):
        self.L = img_as_float(io.imread(imgPath))
        self.row, self.col = self.L.shape[:2]
        self.T_hat = np.max(self.L, axis=2)
        self.dv = firstOrderDerivative(self.row)
        self.dh = firstOrderDerivative(self.col, -1)
        self.vecDD = toeplitizMatrix(self.row * self.col, self.row)
        self.W = self.weightingStrategy()

    def weightingStrategy(self):
        if self.strategy == 2:
            dTv = self.dv @ self.T_hat
            dTh = self.T_hat @ self.dh
            Wv = 1 / (np.abs(dTv) + 1)
            Wh = 1 / (np.abs(dTh) + 1)
            return np.vstack([Wv, Wh])
        else:
            return np.ones((self.row * 2, self.col))

    def __T_subproblem(self, G, Z, u):
        X = G - Z / u
        Xv = X[:self.row, :]
        Xh = X[self.row:, :]
        temp = self.dv @ Xv + Xh @ self.dh
        numerator = fft.fft(vectorize(2 * self.T_hat + u * temp))
        denominator = fft.fft(self.vecDD * u) + 2
        T = fft.ifft(numerator / denominator)
        return exposure.rescale_intensity(np.real(reshape(T, self.row, self.col)), (0, 1), (0.001, 1))

    def __G_subproblem(self, T, Z, u, W):
        dT = self.__derivative(T)
        epsilon = self.alpha * W / u
        X = dT + Z / u
        return np.sign(X) * np.maximum(np.abs(X) - epsilon, 0)

    def __Z_subproblem(self, T, G, Z, u):
        dT = self.__derivative(T)
        return Z + u * (dT - G)

    def __u_subproblem(self, u):
        return u * self.rho

    def __derivative(self, matrix):
        v = self.dv @ matrix
        h = matrix @ self.dh
        return np.vstack([v, h])

    def illumMap(self):
        T = np.zeros((self.row, self.col))
        G = np.zeros((self.row * 2, self.col))
        Z = np.zeros((self.row * 2, self.col))
        u = 1

        for _ in trange(0, self.iterations, leave=False):
            T = self.__T_subproblem(G, Z, u)
            G = self.__G_subproblem(T, Z, u, self.W)
            Z = self.__Z_subproblem(T, G, Z, u)
            u = self.__u_subproblem(u)

        return T ** self.gamma

    def enhance(self):
        self.T = self.illumMap()
        self.R = self.L / np.repeat(self.T[:, :, np.newaxis], 3, axis=2)
        self.R = exposure.rescale_intensity(self.R, (0, 1))
        self.R = img_as_ubyte(self.R)
        return self.R

# ------------------ BATCH VERWERKING ------------------ #

def process_folder(input_root, output_root):
    lime = LIME()

    for root, dirs, files in os.walk(input_root):
        for file in files:
            if not file.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            input_path = os.path.join(root, file)
            rel_path = os.path.relpath(input_path, input_root)
            output_path = os.path.join(output_root, rel_path)

            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            try:
                lime.load(input_path)
                enhanced = lime.enhance()
                Image.fromarray(enhanced).save(output_path)
                print(f"{rel_path}")
            except Exception as e:
                print(f"Fout bij {rel_path}: {e}")


input_root = "data-filtered-rooms"           # map met room1, room2, ...
output_root = "data-lime2"      # waar het resultaat komt

process_folder(input_root, output_root)


#### Een aantal wat simpelere methoden, gebaseerd op OpenCV-documentatie

https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html <br>
https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html

In [ ]:
import cv2
import numpy as np
from skimage import img_as_float
from PIL import Image

def apply_clahe(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

def gamma_correction(img, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, table)

def lime_enhance(img):
    illum_map = np.max(img / 255.0, axis=2)
    illum_map = cv2.GaussianBlur(illum_map, (5, 5), 0)
    enhanced = np.clip(img / (illum_map[..., None] + 1e-3), 0, 255).astype(np.uint8)
    return enhanced

In [ ]:
base_path = "data-filtered-rooms"
output_base = "data-{method}"  # Placeholder

methods = {
    "clahe": apply_clahe,
    "gamma": lambda img: gamma_correction(img, gamma=1.5),
    "lime": lime_enhance
}

for method_name, func in methods.items():
    print(f"Methode: {method_name}")

    for i in range(1, 32):
        room_path = os.path.join(base_path, f"room{i}")
        if not os.path.isdir(room_path):
            print(f"{room_path} bestaat niet, overslaan...")
            continue

        output_path = os.path.join(output_base.format(method=method_name), f"room{i}")
        os.makedirs(output_path, exist_ok=True)

        for filename in os.listdir(room_path):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            input_file = os.path.join(room_path, filename)
            output_file = os.path.join(output_path, filename)

            try:
                # Open als numpy-array
                with Image.open(input_file) as img_pil:
                    img = np.array(img_pil.convert("RGB"))

                # Pas LLIE methode toe
                enhanced = func(img)

                # Opslaan
                Image.fromarray(enhanced).save(output_file)

            except Exception as e:
                print(f"Fout bij {input_file}: {e}")

#### MSRCR code van https://github.com/muggledy/retinex/

In [ ]:
def MultiScaleRetinex(img,sigmas=[15,80,250],weights=None,flag=True):
    '''equal to func retinex_MSR, just remove the outer for-loop. Practice has proven 
       that when MSR used in MSRCR or Gimp, we should add stretch step, otherwise the 
       result color may be dim. But it's up to you, if you select to neglect stretch, 
       set flag as False, have fun'''
    if weights==None:
        weights=np.ones(len(sigmas))/len(sigmas)
    elif not abs(sum(weights)-1)<0.00001:
        raise ValueError('sum of weights must be 1!')
    r=np.zeros(img.shape,dtype='double')
    img=img.astype('double')
    for i,sigma in enumerate(sigmas):
        r+=(np.log(img+1)-np.log(gauss_blur(img,sigma)+1))*weights[i]
    if flag:
        mmin=np.min(r,axis=(0,1),keepdims=True)
        mmax=np.max(r,axis=(0,1),keepdims=True)
        r=(r-mmin)/(mmax-mmin)*255 #maybe indispensable when used in MSRCR or Gimp, make pic vibrant
        r=r.astype('uint8')
    return r


def simplest_color_balance(img_msrcr,s1,s2):
    '''see section 3.1 in “Simplest Color Balance”(doi: 10.5201/ipol.2011.llmps-scb). 
    Only suitable for 1-channel image'''
    sort_img=np.sort(img_msrcr,None)
    N=img_msrcr.size
    Vmin=sort_img[int(N*s1)]
    Vmax=sort_img[int(N*(1-s2))-1]
    img_msrcr[img_msrcr<Vmin]=Vmin
    img_msrcr[img_msrcr>Vmax]=Vmax
    return (img_msrcr-Vmin)*255/(Vmax-Vmin)

def retinex_MSRCR(img,sigmas=[12,80,250],s1=0.01,s2=0.01):
    '''r=βlog(αI')MSR, I'=I/∑I, I is one channel of image, ∑I is the sum of all channels, 
       C:=βlog(αI') is named as color recovery factor. Last we improve previously used 
       linear stretch: MSRCR:=r, r=G[MSRCR-b], then doing linear stretch. In practice, it 
       doesn't work well, so we take another measure: Simplest Color Balance'''
    alpha=125
    img=img.astype('double')+1 #
    csum_log=np.log(np.sum(img,axis=2))
    msr=MultiScaleRetinex(img-1,sigmas) #-1
    r=(np.log(alpha*img)-csum_log[...,None])*msr
    #beta=46;G=192;b=-30;r=G*(beta*r-b) #deprecated
    #mmin,mmax=np.min(r),np.max(r)
    #stretch=(r-mmin)/(mmax-mmin)*255 #linear stretch is unsatisfactory
    for i in range(r.shape[-1]):
        r[...,i]=simplest_color_balance(r[...,i],0.01,0.01)
    return r.astype('uint8')


def get_gauss_kernel(sigma,dim=2):
    '''1D gaussian function: G(x)=1/(sqrt{2π}σ)exp{-(x-μ)²/2σ²}. Herein, μ:=0, after 
       normalizing the 1D kernel, we can get 2D kernel version by 
       matmul(1D_kernel',1D_kernel), having same sigma in both directions. Note that 
       if you want to blur one image with a 2-D gaussian filter, you should separate 
       it into two steps(i.e. separate the 2-D filter into two 1-D filter, one column 
       filter, one row filter): 1) blur image with first column filter, 2) blur the 
       result image of 1) with the second row filter. Analyse the time complexity: if 
       m&n is the shape of image, p&q is the size of 2-D filter, bluring image with 
       2-D filter takes O(mnpq), but two-step method takes O(pmn+qmn)'''
    ksize=int(np.floor(sigma*6)/2)*2+1 #kernel size("3-σ"法则) refer to 
    #https://github.com/upcAutoLang/MSRCR-Restoration/blob/master/src/MSRCR.cpp
    k_1D=np.arange(ksize)-ksize//2
    k_1D=np.exp(-k_1D**2/(2*sigma**2))
    k_1D=k_1D/np.sum(k_1D)
    if dim==1:
        return k_1D
    elif dim==2:
        return k_1D[:,None].dot(k_1D.reshape(1,-1))

def gauss_blur_original(img,sigma):
    '''suitable for 1 or 3 channel image'''
    row_filter=get_gauss_kernel(sigma,1)
    t=cv2.filter2D(img,-1,row_filter[...,None])
    return cv2.filter2D(t,-1,row_filter.reshape(1,-1))

def gauss_blur_recursive(img,sigma):
    '''refer to “Recursive implementation of the Gaussian filter”
       (doi: 10.1016/0165-1684(95)00020-E). Paper considers it faster than 
       FFT(Fast Fourier Transform) implementation of a Gaussian filter. 
       Suitable for 1 or 3 channel image'''
    pass

def gauss_blur(img,sigma,method='original'):
    if method=='original':
        return gauss_blur_original(img,sigma)
    elif method=='recursive':
        return gauss_blur_recursive(img,sigma)
    

base_path = "data-filtered-rooms"
output_base = "data-msrcr"


import os
import cv2
import numpy as np

for i in range(1, 32):
    room_path = os.path.join(base_path, f"room{i}")
    if not os.path.isdir(room_path):
        print(f"{room_path} bestaat niet, overslaan...")
        continue

    output_path = os.path.join(output_base.format(method="MSRCR"), f"room{i}")
    os.makedirs(output_path, exist_ok=True)

    for filename in os.listdir(room_path):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        input_file = os.path.join(room_path, filename)
        output_file = os.path.join(output_path, filename)

        try:
            # Lees afbeelding in met OpenCV en converteer naar RGB
            img_bgr = cv2.imread(input_file)
            if img_bgr is None:
                raise ValueError("Afbeelding kon niet worden ingelezen")
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

            # Pas LLIE-methode toe
            enhanced_rgb = retinex_MSRCR(img_rgb)

            # Converteer terug naar BGR voor correcte opslag met OpenCV
            enhanced_bgr = cv2.cvtColor(enhanced_rgb, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_file, enhanced_bgr)
            print("gelukt")

        except Exception as e:
            print(f"Fout bij {input_file}: {e}")


gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt
gelukt

In [ ]:
predict_falls("data-clahe", 0.5)

In [ ]:
predict_falls("data-gamma", 0.05)

In [ ]:
predict_falls("data-lime", 0.5)

In [ ]:
predict_falls("data-lime2", 0.5)

In [ ]:
predict_falls("data-msrcr", 0.5)

#### Deep learning-resultaten, de afbeeldingen zijn gegenereerd in ZeroDceLite/finetune.ipynb (door het niet gefinetunede model in te laten en te voorspellen) en RetinexNet/test.ipynb

In [ ]:
predict_falls("data-retinexnet", 0.05)

In [ ]:
predict_falls("data-zerodce", 0.5)

In [ ]:
predict_falls("data-zerodce-lite", 0.5)

#### NIQE-metric berekenen via BasicSR

In [ ]:
import os
import cv2
import numpy as np
from basicsr.metrics.niqe import calculate_niqe

def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))

def calculate_avg_niqe_in_data_folder(data_folder):
    niqe_scores = []

    # Doorloop submappen die beginnen met "room"
    for room_name in os.listdir(data_folder):
        room_path = os.path.join(data_folder, room_name)
        if os.path.isdir(room_path) and room_name.lower().startswith("room"):
            for filename in os.listdir(room_path):
                if is_image_file(filename):
                    img_path = os.path.join(room_path, filename)
                    img = cv2.imread(img_path)
                    if img is None:
                        continue
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    h, w = img.shape
                    if h < 128 or w < 128:
                        img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
                    try:
                        score = calculate_niqe(img, crop_border=0)
                        niqe_scores.append(score)
                    except Exception as e:
                        print(f"Fout bij berekenen NIQE voor {img_path}: {e}")

    if niqe_scores:
        return np.mean(niqe_scores)
    else:
        return None

base_dir="."
for entry in os.listdir(base_dir):
    if "data" in entry.lower():
        data_folder_path = os.path.join(base_dir, entry)
        if os.path.isdir(data_folder_path):
            print(f"\nVerwerken van datasetmap: {entry}")
            avg_niqe = calculate_avg_niqe_in_data_folder(data_folder_path)
            if avg_niqe is not None:
                print(f"Gemiddelde NIQE-score voor '{entry}': {avg_niqe:.4f}")
            else:
                print(f"Geen geldige afbeeldingen gevonden in '{entry}'")



Verwerken van datasetmap: data-archief
Geen geldige afbeeldingen gevonden in 'data-archief'

Verwerken van datasetmap: data-clahe
Gemiddelde NIQE-score voor 'data-clahe': 4.3612

Verwerken van datasetmap: data-filtered-rooms
Gemiddelde NIQE-score voor 'data-filtered-rooms': 5.3462

Verwerken van datasetmap: data-gamma
Gemiddelde NIQE-score voor 'data-gamma': 4.6719

Verwerken van datasetmap: data-lime
Gemiddelde NIQE-score voor 'data-lime': 4.2360

Verwerken van datasetmap: data-lime2
Gemiddelde NIQE-score voor 'data-lime2': 4.2768

Verwerken van datasetmap: data-mirnet
Gemiddelde NIQE-score voor 'data-mirnet': 4.0885

Verwerken van datasetmap: data-msrcr
Gemiddelde NIQE-score voor 'data-msrcr': 4.1169

Verwerken van datasetmap: data-retinex
Gemiddelde NIQE-score voor 'data-retinex': 5.3965

Verwerken van datasetmap: data-retinexnet
Gemiddelde NIQE-score voor 'data-retinexnet': 4.0092

Verwerken van datasetmap: data-uretinex
Geen geldige afbeeldingen gevonden in 'data-uretinex'

Verwe